In [ ]:
!pip install --upgrade --user google-cloud-aiplatform>=1.29.0 google-cloud-storage

In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
# get project ID
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

In [2]:
# define project information
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]"

# generate a unique id for this session
from datetime import datetime
UID = datetime.now().strftime("%m%d%H%M")

In [3]:
! gcloud services enable compute.googleapis.com aiplatform.googleapis.com storage.googleapis.com --project {PROJECT_ID}

Operation "operations/acat.p2-26258447032-578c8f96-32a8-41a3-82a6-3e629d06f976" finished successfully.


In [4]:
BUCKET_URI = f"gs://{PROJECT_ID}-vs-quickstart-{UID}"

In [5]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI
! gsutil cp "gs://github-repo/data/vs-quickstart/product-embs.json" $BUCKET_URI

Creating gs://vertex-ai-poc-416620-vs-quickstart-03082027/...
Copying gs://github-repo/data/vs-quickstart/product-embs.json [Content-Type=application/json]...
- [1 files][ 79.3 MiB/ 79.3 MiB]                                                
Operation completed over 1 objects/79.3 MiB.                                     


In [6]:
! gsutil cp "gs://github-repo/data/vs-quickstart/product-embs.json" . # for query tests

Copying gs://github-repo/data/vs-quickstart/product-embs.json...
- [1 files][ 79.3 MiB/ 79.3 MiB]                                                
Operation completed over 1 objects/79.3 MiB.                                     


In [7]:
# init the aiplatform package
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [8]:
# create Index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name = f"vs-quickstart-index-{UID}",
    contents_delta_uri = BUCKET_URI,
    dimensions = 768,
    approximate_neighbors_count = 10,
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/26258447032/locations/us-central1/indexes/3487444175119450112/operations/4925508661586952192
MatchingEngineIndex created. Resource name: projects/26258447032/locations/us-central1/indexes/3487444175119450112
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/26258447032/locations/us-central1/indexes/3487444175119450112')


In [9]:
## create `IndexEndpoint`
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name = f"vs-quickstart-index-endpoint-{UID}",
    public_endpoint_enabled = True
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/26258447032/locations/us-central1/indexEndpoints/4912833457182212096/operations/7420502855150206976
MatchingEngineIndexEndpoint created. Resource name: projects/26258447032/locations/us-central1/indexEndpoints/4912833457182212096
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/26258447032/locations/us-central1/indexEndpoints/4912833457182212096')


In [10]:
DEPLOYED_INDEX_ID = f"vs_quickstart_deployed_{UID}"

In [ ]:
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(
    index = my_index, deployed_index_id = DEPLOYED_INDEX_ID
)

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/26258447032/locations/us-central1/indexEndpoints/4912833457182212096
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/26258447032/locations/us-central1/indexEndpoints/4912833457182212096/operations/7482427350026551296


In [12]:
import json

# build dicts for product names and embs
product_names = {}
product_embs = {}
with open('product-embs.json') as f:
    for l in f.readlines():
        p = json.loads(l)
        id = p['id']
        product_names[id] = p['name']
        product_embs[id] = p['embedding']

In [14]:
 # Get the embedding for ID 6523 "cloudveil women's excursion short"
query_emb = product_embs['6523']

In [15]:
# run query
response = my_index_endpoint.find_neighbors(
    deployed_index_id = DEPLOYED_INDEX_ID,
    queries = [query_emb],
    num_neighbors = 10
)

# show the results
for idx, neighbor in enumerate(response[0]):
    print(f"{neighbor.distance:.2f} {product_names[neighbor.id]}")

1.00 cloudveil women's excursion short
0.82 quiksilver womens cruiser short
0.80 xcvi women's alisal short
0.80 cloudveil men's kahuna short
0.78 ibex women's gozo short
0.78 sanctuary clothing women's coquette short
0.78 sunner women's collins printed short
0.77 hurley lowrider cargo 2.5 short - women's
0.77 stitch's women's fox knee length short
0.77 sanctuary clothing women's passenger skirt


In [16]:

# wait for a confirmation
input("Press Enter to delete Index Endpoint, Index and Cloud Storage bucket:")

# delete Index Endpoint
my_index_endpoint.undeploy_all()
my_index_endpoint.delete(force = True)

# delete Index
my_index.delete()

# delete Cloud Storage bucket
! gsutil rm -r {BUCKET_URI}

Press Enter to delete Index Endpoint, Index and Cloud Storage bucket: 


Undeploying MatchingEngineIndexEndpoint index_endpoint: projects/26258447032/locations/us-central1/indexEndpoints/4912833457182212096
Undeploy MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/26258447032/locations/us-central1/indexEndpoints/4912833457182212096/operations/2197928176330473472
MatchingEngineIndexEndpoint index_endpoint undeployed. Resource name: projects/26258447032/locations/us-central1/indexEndpoints/4912833457182212096
Deleting MatchingEngineIndexEndpoint : projects/26258447032/locations/us-central1/indexEndpoints/4912833457182212096
Delete MatchingEngineIndexEndpoint  backing LRO: projects/26258447032/locations/us-central1/indexEndpoints/4912833457182212096/operations/1657496221046013952
MatchingEngineIndexEndpoint deleted. . Resource name: projects/26258447032/locations/us-central1/indexEndpoints/4912833457182212096
Deleting MatchingEngineIndex : projects/26258447032/locations/us-central1/indexes/3487444175119450112
Delete MatchingEngineIndex  backing